> References: 
1. https://cs231n.github.io/convolutional-networks/#overview
1. https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/convolution.html
2. https://www.reddit.com/r/MLQuestions/comments/8no4xe/anyone_familiar_with_how_tensorflow_or_pytorch/
3. https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/making_faster.html

# Convolution in Mathematics
- Convolution is a mathematical operation that does the integral of the product of 2 functions(signals), with one of the signals flipped. 
- Equation: 
$$ \begin{split} y[n_1, n_2] &= \sum_{k_1 = -\infty}^{\infty} \sum_{k_2 = -\infty}^{\infty} x[k_1, k_2] h [n_1 - k_1, n_2 - k_2] \\ 
&= x[n_1, n_2] * h[n_1, n_2] \\
&= h[n_1, n_2] * x[n_1, n_2]
\end{split} $$

## Simple example in matlab and python
- x = [0, 1 2, 3, 4];
- w = [1 -1 2];
- $y = x*w$

In [4]:
import numpy as np
x = np.array(range(0,5))
w = np.array([1, -1,2])
y = np.convolve(x,w)
print ('x * w = {}'.format(y))

x * w = [0 1 1 3 5 2 8]


## Doing by hand
To understand better the concept of convolution let's do the example above by hand. Basically we're going to convolve 2 signals (x,w). The first thing is to flip W horizontally (Or rotate to left 180 degrees)

<img src="../files/Conv1d_Manual.png" alt="Conv1d_Manual" width="600"/>

## Transforming convolution to computation graph
In order to calculate partial derivatives of every nodes inputs and parameters, it's easier to transform the operation to a computational graph. Here I'm going to transform the previous 1D convolution, but this can be extended to 2D convolution as well. 

<img src="../files/Conv1d_Manual_symbolic.png" alt="Conv1d_Manual_symbolic" width="900"/>

Here our graph will be created on the valid cases where the flipped kernel(weights) will be fully inserted on our input window.  

<img src="../files/Simple_1d_Conv_graph.png" alt="Simple_1d_Conv_graph" width="800"/>
We're going to use this graph in the future to infer the gradients of the inputs (x) and weights (w) of the convolution layer.  

## Implementing convolution operation
- **Spatial arrangement:** We can compute the spatial size of the output volume as a function of the input volume size (W), the receptive field size of the Conv Layer neurons (F), the stride with which they are applied (S), and the amount of zero padding used (P) on the border. You can convince yourself that the correct formula for calculating how many neurons “fit” is given by $(W−F+2P)/S+1$. For example for a 7x7 input and a 3x3 filter with stride 1 and pad 0 we would get a 5x5 output.
- The example below will convolve a $5\times 5 \times 3$ ($W \times H \times 3$) input, with a conv layer with the following parameters Stride=2, Pad=1, F=3 (3x3 kernel), and K=2 (two filters).

- Our input has 3 channels, so we need a 3x3x3 kernel weight. We have 2 filters (K=2) so we will have 2 output activations at the end. Also we can calculate the size of these two outputs to be: (5 - 3 + 2)/2 + 1 = 3.

- So we will get a final output volume of size (3x3x2).

**Note:** The code below (vanilla version) cannot be used in real life because it will be slow but its good for a basic understanding. Usually **deep learning libraries** do the convolution as **matrix multiplication**, using the **im2col/col2im** method.

## Numpy examples:

To make the discussion above more concrete, lets express the same ideas but in code and with a specific example. Suppose that the input volume is a numpy array $X$. Then:
- A depth column (or a fibre) at position (x,y) would be the activations $X[x,y,:]$.
- A depth slice, or equivalently an activation map at depth $d$ would be the activations $X[:,:,d]$.

**Conv Layer Example.** Suppose that the input volume X has shape `X.shape: (11,11,4)`. Suppose further that we use no zero padding ($P=0$), that the filter size is $F=5$, and that the stride is $S=2$. The output volume would therefore have spatial size `(11-5)/2+1 = 4`, giving a volume with width and height of 4. The activation map in the output volume (call it $V$), would then look as follows (only some of the elements are computed in this example):

- `V[0,0,0] = np.sum(X[:5,:5,:] * W0) + b0`
- `V[1,0,0] = np.sum(X[2:7,:5,:] * W0) + b0`
- `V[2,0,0] = np.sum(X[4:9,:5,:] * W0) + b0`
- `V[3,0,0] = np.sum(X[6:11,:5,:] * W0) + b0`   

Remember that in numpy, the operation `*` above denotes **elementwise multiplication** between the arrays. Notice also that the weight vector $W0$ is the weight vector of that neuron and $b0$ is the bias. Here, $W0$ is assumed to be of shape `W0.shape: (5,5,4)`, since the filter size is 5 and the depth of the input volume is 4. Notice that at each point, we are computing the ** dot product ** as seen before in ordinary neural networks. Also, we see that we are using the same weight and bias (due to **parameter sharing**), and where the dimensions along the width are increasing in steps of 2 (i.e. the `stride`). To construct a second activation map in the output volume, we would have:

- `V[0,0,1] = np.sum(X[:5,:5,:] * W1) + b1`
- `V[1,0,1] = np.sum(X[2:7,:5,:] * W1) + b1`
- `V[2,0,1] = np.sum(X[4:9,:5,:] * W1) + b1`
- `V[3,0,1] = np.sum(X[6:11,:5,:] * W1) + b1`
- `V[0,1,1] = np.sum(X[:5,2:7,:] * W1) + b1 (example of going along y)`
- `V[2,3,1] = np.sum(X[4:9,6:11,:] * W1) + b1 (or along both)`

where we see that we are indexing into the second depth dimension in V (at index 1) because we are computing the second activation map, and that a different set of parameters (W1) is now used. In the example above, we are for brevity leaving out some of the other operations the Conv Layer would perform to fill the other parts of the output array V. Additionally, recall that these activation maps are often followed elementwise through an activation function such as ReLU, but this is not shown here.

## Implementation as Matrix Multiplication.

Note that the convolution operation essentially performs dot products between the filters and local regions of the input. A common implementation pattern of the CONV layer is to take advantage of this fact and formulate the forward pass of a convolutional layer as `one big matrix multiply` as follows:

1. The local regions in the input image are stretched out into columns in an operation commonly called `im2col`. For example, if the input is [227x227x3] and it is to be convolved with 11x11x3 filters at stride 4, then we would take [11x11x3] blocks of pixels in the input and stretch each block into a column vector of size `11*11*3 = 363`. Iterating this process in the input at stride of 4 gives `(227-11)/4+1 = 55` locations along both width and height, leading to an output matrix `X_col` of `im2col` of size [363 x 3025], where every column is a stretched out receptive field and there are `55*55 = 3025` of them in total. Note that since the receptive fields overlap, every number in the input volume may be duplicated in multiple distinct columns.

2. The weights of the CONV layer are similarly stretched out into rows. For example, if there are 96 filters of size [11x11x3] this would give a matrix `W_row` of size [96 x 363].

3. The result of a convolution is now equivalent to performing one large matrix multiply `np.dot(W_row, X_col)`, which evaluates the dot product between every filter and every receptive field location. In our example, the output of this operation would be `[96 x 3025]`, giving the output of the dot product of each filter at each location.
4. The result must finally be reshaped back to its proper output dimension `[55x55x96]`.

This approach has the `downside` that it can use a lot of `memory`, since some values in the input volume are `replicated` multiple times in `X_col`. However, the benefit is that there are many very efficient implementations of Matrix Multiplication that we can take advantage of (for example, in the commonly used [BLAS (Basic Linear Algebra Subprograms)](http://www.netlib.org/blas/) API). Moreover, the same `im2col` idea can be reused to perform the `pooling` operation, which we discuss next.

## 1x1 convolution
As an aside, several papers use `1x1` convolutions, as first investigated by Network in Network. Some people are at first confused to see 1x1 convolutions especially when they come from signal processing background. Normally signals are 2-dimensional so 1x1 convolutions do not make sense (it’s just pointwise scaling). However, in ConvNets this is not the case because one must remember that we operate over `3-dimensional volumes`, and that the filters always extend through the full depth of the input volume. For example, if the input is `[32x32x3]` then doing `1x1` convolutions would effectively be doing `3-dimensional` `dot products` (since the input depth is 3 channels).

# Convolutional Neural Networks (CNNs / ConvNets)
Convolutional Neural Networks are very similar to ordinary Neural Networks from the previous chapter: they are made up of neurons that have learnable weights and biases. Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. The whole network still expresses a single differentiable score function: from the raw image pixels on one end to class scores at the other. And they still have a loss function (e.g. SVM/Softmax) on the last (fully-connected) layer and all the tips/tricks we developed for learning regular Neural Networks still apply.

So what changes? ConvNet architectures make the **explicit assumption** that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.


